In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === load data
makan = pd.read_csv("/content/makanan.csv")

# === parameter dasar
NPM = 19102
np.random.seed(NPM)

budget0 = 1e6 #rupiah

wkenyang_base= 0.8
wrasa_base= 0.7
wsehat_base= 0.9
wbosan_base= 0.2

SAMPLE_NUM= 500
SISA_MININUM= 150e3
SISA_MAKSIMUM= 300e3
bin_sisa= np.arange(SISA_MININUM, SISA_MAKSIMUM, 1000)

# === arrays
sampleid= np.arange(0, SAMPLE_NUM)
skor= np.zeros(SAMPLE_NUM)
sisa= np.zeros(SAMPLE_NUM)
sehat= np.zeros(SAMPLE_NUM)
resp_sehat= np.zeros((SAMPLE_NUM, len(bin_sisa)))
resp_pilih= np.zeros((SAMPLE_NUM, 62))  # 31 hari × 2 makan

# === simulasi
for n in range(SAMPLE_NUM):
    budget=budget0
    skor_kumulatif= 0
    skor_sehat= 0

    for i in range(31): # hari
        # makan pertama
        pilihan= np.random.randint(0, len(makan))
        w_kenyang= np.random.normal(loc=wkenyang_base, scale=0.2)
        w_rasa= np.random.normal(loc=wrasa_base, scale=0.2)
        w_sehat= np.random.normal(loc=wsehat_base, scale=0.4)
        w_bosan= np.random.normal(loc=wbosan_base, scale=0.2)
        skor_kumulatif += (w_kenyang * makan['kenyang'][pilihan]
                          + w_rasa * makan['rasa'][pilihan]
                          + w_sehat * makan['sehat'][pilihan]
                          - w_bosan * makan['bosan'][pilihan])
        skor_sehat += makan['sehat'][pilihan] - 0.5
        budget -= makan['harga'][pilihan]
        resp_pilih[n][i*2]= pilihan

        # makan kedua
        pilihan= np.random.randint(0, len(makan))
        w_kenyang= np.random.normal(loc=wkenyang_base, scale=0.2)
        w_rasa= np.random.normal(loc=wrasa_base, scale=0.2)
        w_sehat= np.random.normal(loc=wsehat_base+0.2, scale=0.4)
        w_bosan= np.random.normal(loc=wbosan_base*2, scale=0.2)
        skor_kumulatif += (w_kenyang * makan['kenyang'][pilihan]
                          + w_rasa * makan['rasa'][pilihan]
                          + w_sehat * makan['sehat'][pilihan]
                          - w_bosan * makan['bosan'][pilihan])
        skor_sehat += makan['sehat'][pilihan] - 0.5
        budget -= makan['harga'][pilihan]
        resp_pilih[n][i*2+1]= pilihan

    sisa[n]= budget
    skor[n]= skor_kumulatif
    sehat[n]= skor_sehat

    idx_raw=(budget-SISA_MININUM)/1000
    if np.isfinite(idx_raw) and 0<=idx_raw<len(bin_sisa):
        idx=int(idx_raw)
        resp_sehat[n][idx]=skor_sehat

# === (3) titik dengan skor skumulatif terbesar (skor) di mana nilai kumulatif kesehatannya positif
mask3=(sehat>0) & (sisa>0)
idx_max_skor=np.argmax(np.where(mask3, skor, -np.inf))
max_skor_value=skor[idx_max_skor]
max_skor_sehat=sehat[idx_max_skor]
max_skor_sisa=sisa[idx_max_skor]

# === (4) tampilkan jadwal makanan sesuai pilihan (resp_pilih)
jadwal_makanan=pd.DataFrame(resp_pilih[idx_max_skor,:],columns=['Index_Makanan'])
jadwal_makanan['Nama_Makanan']=jadwal_makanan['Index_Makanan'].apply(
    lambda x: makan['makanan'][int(x)] if x<len(makan) else None)

# tampilkan di colab
print("=== Jadwal makanan terbaik (index & nama) ===")
display(jadwal_makanan)

print("\nSkor kumulatif terbesar:", max_skor_value)
print("Nilai sehat kumulatif:", max_skor_sehat)
print("Sisa uang bulanan:", max_skor_sisa)


=== Jadwal makanan terbaik (index & nama) ===


,Index_Makanan,Nama_Makanan
0,4.0,cireng
1,11.0,nasi telur
2,12.0,warteg
3,12.0,warteg
4,2.0,mieayam
...,...,...
57,9.0,mi instan
58,7.0,gorengan
59,10.0,roti
60,8.0,capcay



Skor kumulatif terbesar: 88.19157153488203
Nilai sehat kumulatif: 0.6000000000000009
Sisa uang bulanan: nan
